# Loop 6 LB Feedback Analysis

## Key Observations:
1. Grid initialization found ZERO improvements - our ensemble already has optimal grid-based solutions
2. CV = LB perfectly (no distribution shift - expected for deterministic optimization)
3. Target (68.93) is BETTER than LB #1 (71.19) - need innovation beyond public solutions
4. Gap: 84.71 - 68.93 = 15.78 points (18.6% improvement needed)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load submission history
submissions = [
    {'exp': 'exp_000', 'cv': 135.819103, 'lb': 135.819103, 'method': 'baseline'},
    {'exp': 'exp_001', 'cv': 117.281454, 'lb': 117.281454, 'method': 'multiphase'},
    {'exp': 'exp_003', 'cv': 84.901044, 'lb': 84.901044, 'method': 'ensemble_fixed'},
    {'exp': 'exp_004', 'cv': 84.712432, 'lb': 84.712432, 'method': 'extended_opt'},
    {'exp': 'exp_005', 'cv': 84.712432, 'lb': 84.712432, 'method': 'grid_init'},
]

df = pd.DataFrame(submissions)
print("Submission History:")
print(df)

# CV-LB relationship
print(f"\nCV-LB Correlation: {np.corrcoef(df['cv'], df['lb'])[0,1]:.6f}")
print("Perfect CV=LB alignment (expected for deterministic optimization)")

# Progress analysis
print(f"\nProgress:")
print(f"  Starting score: {df['cv'].iloc[0]:.2f}")
print(f"  Current best: {df['cv'].min():.2f}")
print(f"  Improvement: {df['cv'].iloc[0] - df['cv'].min():.2f} ({(df['cv'].iloc[0] - df['cv'].min())/df['cv'].iloc[0]*100:.1f}%)")
print(f"  Target: 68.93")
print(f"  Gap to target: {df['cv'].min() - 68.93:.2f} ({(df['cv'].min() - 68.93)/68.93*100:.1f}% above target)")

In [ ]:
# Analyze what's needed to reach target
target = 68.931058
current = 84.712432

print("=" * 60)
print("GAP ANALYSIS")
print("=" * 60)
print(f"Current score: {current:.6f}")
print(f"Target score: {target:.6f}")
print(f"Gap: {current - target:.6f}")
print(f"Reduction needed: {(current - target)/current * 100:.1f}%")

# What would it take?
print(f"\nTo reach target, we need to reduce score by {current - target:.2f} points")
print(f"This is equivalent to reducing EVERY N's score by {(current - target)/200:.4f} on average")

# Theoretical analysis
print(f"\nTheoretical minimum (perfect packing): ~70.0")
print(f"Target is {target - 70:.2f} above theoretical minimum")
print(f"Current is {current - 70:.2f} above theoretical minimum")
print(f"We need to close {(current - target)/(current - 70) * 100:.1f}% of the gap to theoretical")

In [ ]:
# Load current best submission and analyze per-N scores
import numpy as np

df_sub = pd.read_csv('/home/code/experiments/006_grid_initialization/submission_grid.csv')

# Fast scoring
TX = np.array([0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125])
TY = np.array([0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5])

def strip_s(val):
    s = str(val)
    return float(s[1:] if s.startswith('s') else s)

def score_group_fast(xs, ys, degs):
    n = len(xs)
    if n == 0:
        return float('inf')
    all_x, all_y = [], []
    for i in range(n):
        rad = np.radians(degs[i])
        c, s = np.cos(rad), np.sin(rad)
        px = TX * c - TY * s + xs[i]
        py = TX * s + TY * c + ys[i]
        all_x.extend(px)
        all_y.extend(py)
    all_x, all_y = np.array(all_x), np.array(all_y)
    side = max(all_x.max() - all_x.min(), all_y.max() - all_y.min())
    return side * side / n

# Calculate per-N scores
scores = {}
for n in range(1, 201):
    group = df_sub[df_sub['id'].str.startswith(f'{n:03d}_')]
    if len(group) == n:
        xs = group['x'].apply(strip_s).values
        ys = group['y'].apply(strip_s).values
        degs = group['deg'].apply(strip_s).values
        scores[n] = score_group_fast(xs, ys, degs)

print(f"Total score: {sum(scores.values()):.6f}")
print(f"\nTop 20 worst N values (highest contribution to score):")
worst = sorted(scores.items(), key=lambda x: -x[1])[:20]
for n, score in worst:
    print(f"  N={n:3d}: {score:.6f} (contributes {score/sum(scores.values())*100:.2f}%)")

print(f"\nTop 20 worst contribute: {sum(s for n,s in worst)/sum(scores.values())*100:.1f}% of total score")

In [ ]:
# Analyze efficiency per N
print("\nEfficiency Analysis (lower score = better packing):")
print("="*60)

# Theoretical minimum for each N (assuming perfect square packing)
# For N trees, theoretical minimum side is approximately sqrt(N * tree_area)
# Tree area is approximately 0.35 (rough estimate)
tree_area = 0.35  # approximate

efficiencies = []
for n in range(1, 201):
    if n in scores:
        actual = scores[n]
        # Theoretical: side^2/n = tree_area (if perfectly packed)
        theoretical = tree_area
        efficiency = theoretical / actual
        efficiencies.append((n, actual, theoretical, efficiency))

# Sort by efficiency (worst first)
efficiencies.sort(key=lambda x: x[3])

print("\nWorst efficiency (most room for improvement):")
for n, actual, theoretical, eff in efficiencies[:20]:
    print(f"  N={n:3d}: score={actual:.4f}, theoretical={theoretical:.4f}, efficiency={eff:.2%}")

print("\nBest efficiency (already well-packed):")
for n, actual, theoretical, eff in efficiencies[-10:]:
    print(f"  N={n:3d}: score={actual:.4f}, theoretical={theoretical:.4f}, efficiency={eff:.2%}")

In [ ]:
# What techniques haven't we tried?
print("="*60)
print("TECHNIQUE ANALYSIS")
print("="*60)

techniques_tried = [
    ("C++ bbox3 optimizer", "YES", "Baseline + extended optimization"),
    ("Fix direction rotation", "YES", "Applied in all experiments"),
    ("Backward propagation", "YES", "No opportunities found - monotonic side lengths"),
    ("Ensemble (best-per-N)", "YES", "125 CSV sources combined"),
    ("Grid initialization (zaburo)", "YES", "No improvements over ensemble"),
    ("Extended SA (-n 50000 -r 200)", "YES", "Only 0.19 points improvement"),
]

techniques_not_tried = [
    ("Per-tree rotation optimization", "Individual tree angle tuning"),
    ("Greedy backtracking (blueshyy)", "Beam search for tree removal"),
    ("Parallel SA with OpenMP", "Multi-threaded optimization (71.78 kernel)"),
    ("Fractional translation polish", "Very fine position adjustments"),
    ("Genetic crossover", "Combine tree positions from different solutions"),
    ("Different compaction strategies", "Center-out, boundary-in, etc."),
    ("Access better datasets", "crodoc/santa2025submission has 74.75 solutions"),
]

print("\nTechniques TRIED:")
for name, status, notes in techniques_tried:
    print(f"  [{status}] {name}: {notes}")

print("\nTechniques NOT YET TRIED:")
for name, notes in techniques_not_tried:
    print(f"  [ ] {name}: {notes}")

In [ ]:
# Key insight from kernels
print("="*60)
print("KEY INSIGHTS FROM TOP KERNELS")
print("="*60)

print("""
1. seshurajup/71-78-jit-parallel-sa-c-tpu-96-cores (71.78 score):
   - Uses TPU with 96 cores for massive parallelism
   - Parallel SA with OpenMP
   - Fractional translation for fine-tuning
   - Multiple generations of optimization
   - Key: COMPUTE POWER + BETTER STARTING SOLUTIONS

2. blueshyy/santa-2025-ensemble-sa-greedy-backtracking (ensemble):
   - Greedy backtracking with beam search
   - Removes trees from N to improve N-1
   - Multi-pass optimization
   - Key: SMART TREE REMOVAL STRATEGY

3. crodoc/74-75-backpacking-christmas-trees (74.75 score):
   - Uses crodoc/santa2025submission dataset (better starting solutions!)
   - Backward iteration from N=200 to N=1
   - Key: ACCESS TO BETTER DATASETS

CRITICAL OBSERVATION:
The 71.78 kernel uses datasets from other kernels as input:
- santa-2025-ensemble-sa-greedy-backtracking
- santa-claude

These datasets likely contain solutions BETTER than what we have!
""")

print("\nRECOMMENDED NEXT STEPS:")
print("1. Implement greedy backtracking (blueshyy approach)")
print("2. Implement per-tree rotation optimization")
print("3. Implement fractional translation polish")
print("4. Try to access better starting solution datasets")
print("5. Run longer optimization with more compute")